In [3]:
import os

In [5]:
os.listdir('/drive')

<class 'OSError'>: [Errno 28] Invalid argument: '/drive'

In [1]:
import micropip
await micropip.install("pure_safetensors")

In [2]:
import numpy as np

a = np.random.rand(10)

In [8]:
a

array([0.2631237 , 0.55841543, 0.71469086, 0.42082077, 0.95911251,
       0.60293777, 0.75385513, 0.01320562, 0.95982688, 0.12350643])

In [21]:
[x for x in range(-5, 5)]

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4]

In [22]:
testarr = [np.triu(np.ones((10,10)), k=i) for i in range(-5, 5)]

In [11]:
b = np.ones((5,5), dtype=np.float16)
print(b)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [25]:
d = [np.power(np.arange(6), i) for i in range(2,4)]

In [26]:
obj = {"foo": a, "lst": testarr, "nest": { "bar": b, "mm": d } }

In [27]:
obj

{'foo': array([0.2631237 , 0.55841543, 0.71469086, 0.42082077, 0.95911251,
        0.60293777, 0.75385513, 0.01320562, 0.95982688, 0.12350643]),
 'lst': [array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.]]),
  array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 

In [4]:
def dict_generator(indict, pre=None):
    pre = pre[:] if pre else []
    if isinstance(indict, dict):
        for key, value in indict.items():
            if isinstance(value, dict):
                for d in dict_generator(value, pre + [key]):
                    yield d
            elif isinstance(value, list):
                for i, v in enumerate(value):
                    for d in dict_generator(v, pre + [key, i]):
                        yield d
            else:
                yield {"path": pre + [key], "val": value} #yield pre + [key, value]
    else:
        yield {"path": pre, "val": indict} #yield pre + [indict]

def generate_name_frompath(p):
    new_p = []
    for x in p:
        if isinstance(x, int):
            new_p.append("[{i}]".format(i=x))
        else:
            new_p.append(x)
    return ".".join(new_p)

def pack_flattened_map(manifest, original_map):
    new_map = {}
    for item in manifest:
        ptr = original_map
        for p in item["path"]:
            ptr = ptr[p]
        new_map[generate_name_frompath(item["path"])] = ptr
    return new_map

def pack_flattened_map_direct(m):
    new_map = {}
    for item in dict_generator(m):
        new_map[generate_name_frompath(item["path"])] = item["val"]
    return new_map


In [5]:
import pure_safetensors as sft

In [6]:
?sft.SafeTensors

Init signature:
sft.SafeTensors(
    file: <class 'IO'>,
    mode: str,
    sparse: bool = None,
    use_mmap: bool = NOTHING,
    do_not_initialize_map=False,
    *,
    settings: 'Settings' = NOTHING,
) -> None
Docstring:     
This object is not thread-safe, and safetensor file structure should not be
modified by different threads or processes at the same time.

Resizing the file may result in a crash:
https://stackoverflow.com/questions/19545949/detect-that-mmap-ed-file-has-been-truncated

Parameters
----------
file: str, file-like, or Path
    File to operate on.
mode: {"r", "r+", "x+", "w+", "c"}
    r: Open existing file for reading only.
    r+: Open existing file for reading and writing.
    w+: Create or overwrite existing file for reading and writing.
    c: Copy-on-write: assignments affect data in memory, but changes are
       not saved to disk. The file on disk is read-only.
sparse: bool or None, optional
    If True, make the file sparse. If False, then don't. If None, t

In [31]:
with sft.SafeTensors("test.safetensors", "w+", use_mmap=False) as st:
    arr = st.as_numpy()
    arr.update(pack_flattened_map_direct(obj))

In [35]:
test_readback = {}
with sft.SafeTensors("test.safetensors", "r", use_mmap=False) as st:
    arr = st.as_numpy()
    for k, v in arr.items():
        test_readback[k] = v

In [36]:
test_readback

{'lst.[0]': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.]]),
 'lst.[1]': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0

In [44]:
import base64

with open("test.safetensors", "rb") as in_file:
    with open("test.safetensors.base64.txt", "wb") as out_file:
        base64.encode(in_file, out_file)

In [42]:
?base64.decode

Signature: base64.decode(input, output)
Docstring: Decode a file; input and output are binary files.
File:      /lib/python311.zip/base64.py
Type:      function


In [45]:
import io

with open("reload.safetensors.base64.txt", "r") as f:
    mybase = f.read()
    tmpio = io.BytesIO(base64.b64decode(mybase))
    test_readback2 = {}
    with sft.SafeTensors(tmpio, "r", use_mmap=False) as st:
        arr = st.as_numpy()
        for k, v in arr.items():
            test_readback2[k] = v

In [46]:
test_readback2

{'lst.[0]': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.]]),
 'lst.[1]': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0

In [7]:
import regex as re

def parse_nested_path(path_str):
    def convert_array_index(s):
        m = re.match("^\[(\d+)\]$", s)
        if m:
            return int(m.groups(1)[0])
        else:
            return s
    
    return [convert_array_index(x) for x in path_str.split('.')]

parse_nested_path("blocks.[2].n_attn.[03].w")

['blocks', 2, 'n_attn', 3, 'w']

In [10]:
await micropip.install("ndicts")

from ndicts import NestedDict


def read_safetensor_raw(filename):
    result = NestedDict({})
    with sft.SafeTensors(filename, "r", use_mmap=False) as st:
        arr = st.as_numpy()
        for k, v in arr.items():
            result[tuple(parse_nested_path(k))] = v
    return result.to_dict()

In [11]:
test_new_read_nested = read_safetensor_raw("test.safetensors")

In [12]:
test_new_read_nested

{'lst': {0: array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.]]),
  1: array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
         [0., 0.